# 패키지, 데이터 로드

In [1]:
# 패키지 로드 
import pandas as pd 
from bs4 import BeautifulSoup  # html 파싱 
import re  # 정규표현식

In [2]:
# 데이터 로드 
file_path = '../../../data/mirae/2023BigFesta_NewsDataset_202302_5.xlsx'  
news = pd.read_excel(file_path)
news.head() # 확인 

,Unnamed: 0,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,9332,202302,20230206_084610,인포스탁,경제,코스닥 프로그램 매매현황(20거래일),53.89,NaN,NaN,"<style type=""text/css"">body,td,select,input,fo..."
1,9333,202302,20230206_085020,인포스탁,기술,지난주 자사주 매수/매도 현황(자사주 체결수량 순),43.27,NaN,NaN,"<style type=""text/css"">body,td,select,input,fo..."
2,9334,202302,20230206_090019,인포스탁,기술,NAVER(035420) 하락 출발 -1.79%,60.22,NAVER,LINE| 일본| 광고| 쇼핑| 금융| 증권사| 인터넷| 라인| 커머스| 디스플레이,"<style type=""text/css"">body,td,select,input,fo..."
3,9335,202302,20230206_090100,인포스탁,기술,"엔피디(198080) 급등 출발 +12.37%, 8거래일 연속 상승, 52주 신고가",65.66,엔피디,OLED,"<style type=""text/css"">body,td,select,input,fo..."
4,9336,202302,20230206_090152,인포스탁,경제,"동원산업(006040) 급등 출발 +15.11%, VI발동",65.80,동원산업,참치| 유통,"<style type=""text/css"">body,td,select,input,fo..."


In [3]:
news.drop('Unnamed: 0',axis = 1, inplace = True) # 불필요한 열 삭제 
news.head() # 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,202302,20230206_084610,인포스탁,경제,코스닥 프로그램 매매현황(20거래일),53.89,NaN,NaN,"<style type=""text/css"">body,td,select,input,fo..."
1,202302,20230206_085020,인포스탁,기술,지난주 자사주 매수/매도 현황(자사주 체결수량 순),43.27,NaN,NaN,"<style type=""text/css"">body,td,select,input,fo..."
2,202302,20230206_090019,인포스탁,기술,NAVER(035420) 하락 출발 -1.79%,60.22,NAVER,LINE| 일본| 광고| 쇼핑| 금융| 증권사| 인터넷| 라인| 커머스| 디스플레이,"<style type=""text/css"">body,td,select,input,fo..."
3,202302,20230206_090100,인포스탁,기술,"엔피디(198080) 급등 출발 +12.37%, 8거래일 연속 상승, 52주 신고가",65.66,엔피디,OLED,"<style type=""text/css"">body,td,select,input,fo..."
4,202302,20230206_090152,인포스탁,경제,"동원산업(006040) 급등 출발 +15.11%, VI발동",65.80,동원산업,참치| 유통,"<style type=""text/css"">body,td,select,input,fo..."


# 키워드 지정
- 아래의 키워드를 뉴스 본문 html 이 저장된 변수인 'CONTENT'변수에 2번 이상 포함하지 않을 경우 엔터테인먼트와 무관한 기사로 판단하여 제거한다.

In [4]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = news[news['CONTENT'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)]

# 특정 키워드가 2번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 2)]


In [5]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3906 entries, 1 to 242983
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NEWS_YM      3906 non-null   int64  
 1   DATE_TIME    3906 non-null   object 
 2   WRITER       3906 non-null   object 
 3   SUBCATEGORY  3906 non-null   object 
 4   TITLE        3905 non-null   object 
 5   IMPORTANCE   3906 non-null   float64
 6   ITEM_NAME    1715 non-null   object 
 7   TAG_LIST     2595 non-null   object 
 8   CONTENT      3906 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 305.2+ KB


# 데이터 저장

In [6]:
# 데이터 저장
file_path = '../../../data/News/news_1filtered_202302.xlsx'  
filtered_df.to_excel(file_path, index=False)